In [1]:
from dotenv import load_dotenv
from google.adk.agents.llm_agent import Agent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from google.adk.tools import agent_tool

In [2]:
load_dotenv(override=True)

True

In [3]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_llm_model = os.getenv("GEMINI_LLM_MODEL")

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

sales_agent1 = Agent(
    name="Professional_Sales_Agent",
    model=gemini_llm_model,
    description=("You write professional, serious cold emails"),
    instruction=(instructions1)
)
sales_agent1_tool = agent_tool.AgentTool(agent=sales_agent1) # Wrap the agent

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

sales_agent2 = Agent(
        name="Engaging_Sales_Agent",
        model=gemini_llm_model,
        description=("You write witty, engaging cold emails that are likely to get a response"),
        instruction=(instructions2)
)
sales_agent2_tool = agent_tool.AgentTool(agent=sales_agent2) # Wrap the agent

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."
sales_agent3 = Agent(
        name="Busy_Sales_Agent",
        model=gemini_llm_model,
        description=("You write concise, to the point cold emails."),
        instruction=(instructions3)
)
sales_agent3_tool = agent_tool.AgentTool(agent=sales_agent3) # Wrap the agent

In [6]:
best_email_picker_instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only."

best_email_picker = Agent(
    name="sales_picker",
    description=("You pick the best cold sales email from a set of provided emails"),
    instruction=(best_email_picker_instructions),
    model=gemini_llm_model,
)

best_email_picker_agent_tool = agent_tool.AgentTool(agent=best_email_picker)


In [7]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

subject_writer = Agent(
    name="Email_subject_writer", 
    description=("You write the subject of the email based on the email body provided"),
    instruction=(subject_instructions), 
    model=gemini_llm_model
    )

subject_writer_agent_tool = agent_tool.AgentTool(agent=subject_writer)

In [8]:
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_converter = Agent(
    name="HTML_email_body_converter", 
    description=("You convert the email into HTML format"),
    instruction=(html_instructions), 
    model=gemini_llm_model
    )

html_converter_agent_tool = agent_tool.AgentTool(agent=html_converter)


In [9]:
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = os.getenv("FROM_EMAIL")
    to_email = os.getenv("TO_EMAIL")
    from_email = Email(from_email)  # Change to your verified sender
    to_email = To(to_email)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
tools = [
    sales_agent1_tool, 
    sales_agent2_tool, 
    sales_agent3_tool, 
    best_email_picker_agent_tool,
    subject_writer_agent_tool,
    html_converter_agent_tool,
    send_html_email]

tools

 <function __main__.send_html_email(subject: str, html_body: str) -> Dict[str, str]>]

In [11]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Use the email_picker_agent_tool to Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Create the subject line: Use the subject_writer_agent_tool to write the subject of the email chosen.

4. Convert to html email format: Use the html_converter_agent_tool to convert the email to html format
 
5. Use the send_html_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

sales_manager = Agent(
    name="Sales_Manager", 
    instruction=(sales_manager_instructions), 
    description=("Your goal is to find the single best cold sales email using the sales_agent tools, pick the best email, get the subject of the email, convert to html format and send the email"),
    tools=tools, 
    model=gemini_llm_model,
    )

In [12]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

session_service = InMemorySessionService()
APP_NAME = "Cold_Email_App"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

Session created: App='Cold_Email_App', User='user_1', Session='session_001'


In [13]:
# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=sales_manager, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Runner created for agent 'Sales_Manager'.


In [14]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [15]:
async def run_conversation():
    message = "Send out a cold sales email addressed to Dear CEO from Alice"
    await call_agent_async(
        message,
        runner=runner,
        user_id=USER_ID,
        session_id=SESSION_ID
        )

In [16]:
await run_conversation()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



>>> User Query: Send out a cold sales email addressed to Dear CEO from Alice


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'request': 'cold sales email addressed to Dear CEO from Alice'
    },
    id='adk-5e17dc13-fa5a-49ee-bfe2-830c6dce73cd',
    name='Professional_Sales_Agent'
  ),
  thought_signature=b'\n\xf3\x05\x01T\xa8\\\xee0\xd1Z\x94~>\xab(\xb0A6\x8ehoh\xb3^v\xa7r\xb7\xcf\x97B0\x90\x05u\xa5\x89z\xb2\x01\xf5TW\xfa\xad0\xcaX\xaf\x1c\xce\xeb\x85\x191\n\x8d\x85\xe8\r\x82( g\x19\x01\xe9@\xb0\xa8\xa1\x91l X\xe6u\xe3G4b&A\xf8Y\x9a\t5\t\xa3}\xde\x8abD\xab...'
), Part(
  function_call=FunctionCall(
    args={
      'request': 'cold sales email addressed to Dear CEO from Alice'
    },
    id='adk-2e9ab01f-37b5-402a-a8ea-22271c10128a',
    name='Engaging_Sales_Agent'
  )
), Part(
  function_call=FunctionCall(
    args={
      'request': 'cold sales email addressed to Dear CEO from Alice'
    },
    id='adk-d991e632-e546-49d4-a09d-92869b13c11e',
    name='Busy_Sales_Agent'
  )
)] role

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


...

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-22fb5dff-a68e-426d-8b73-c2d19532cd08',
    name='HTML_email_body_converter',
    response={
      'result': """```html
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>ComplAI - Your SOC2 Co-pilot</title>
<style type="text/css">
  body {
    margin: 0;
    padding: 0;
    background-color: #f6f6f6;
    font-family: Arial, sans-serif;
    line-height: 1.6;
    color: #333333;
  }
  .container {
    max-width: 600px;
    margin: 20px auto;
    background-color: #ffffff;
    padding: 30px;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.05);
  }
  p {
    margin-top: 0;
    margin-bottom: 20px;
  }
  strong {
    font-weight: bold;
    color: #0056b3; /* A nice blue for emphasis */
  }
  em {
    font-style: 

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'html_body': """<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>ComplAI - Your SOC2 Co-pilot</title>
<style type="text/css">
  body {
    margin: 0;
    padding: 0;
    background-color: #f6f6f6;
    font-family: Arial, sans-serif;
    line-height: 1.6;
    color: #333333;
  }
  .container {
    max-width: 600px;
    margin: 20px auto;
    background-color: #ffffff;
    padding: 30px;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.05);
  }
  p {
    margin-top: 0;
    margin-bottom: 20px;
  }
  strong {
    font-weight: bold;
    color: #0056b3; /* A nice blue for emphasis */
  }
  em {
    font-style: italic;
  }
  .signature {
    margin-top: 30px;
    font-size: 0.9em;
    color: #555555;
  }
  .tagline 

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-17285d84-1276-473c-a8d4-3c0188a600df',
    name='send_html_email',
    response={
      'status': 'success'
    }
  )
)] role='user'


  [Event] Author: Sales_Manager, Type: Event, Final: True, Content: parts=[Part(
  text='The cold sales email has been successfully sent.',
  thought_signature=b'\n\xe9\x03\x01T\xa8\\\xee.\x190\xd8\xa0\x05\x99\xe9\xca!\x0f\x9a\xce(\x98\x0cP\x95\xa6\xec\xca\xbc\x1fQ\xa8t*\r}\xc6\xa6\xfa\xbb\xd7;K\xcbkX6y\x19\x96\x83\xb4\xa0\x7f\xe1\xd0"\x9an\x02\xde\xf2\x1c^\x17Z>\xfd\x8e\x08\xadVT\xa7\x8b\xe434\x16D\x81\xea\xf8\xb4a\xdb+Su|7\x86H0\xf3\xe5...'
)] role='model'
<<< Agent Response: The cold sales email has been successfully sent.
